In [1]:
import threading
import os
import json
import asyncio
import aiohttp

HOST = os.getenv('HOST', 'localhost')
PORT = int(os.getenv('PORT', 8080))

URL = f'http://{HOST}:{PORT}/ws'

async def client(file_count):
    session = aiohttp.ClientSession()
    async with session.ws_connect(URL) as ws:

        await ws.send_str(str(file_count))
        
        async for msg in ws:
            data_received = msg.data
            # Convert received data to a Python dictionary from JSON format
            data_received = json.loads(data_received.replace("'", '"'))
            # Extract the filename to save from the received data
            filename_to_save = data_received["filename"]
            # Convert the content to bytes for writing to a file
            file_content = bytes(data_received["content"])
            # Write the content to a file with the specified filename
            with open(f"{filename_to_save}", 'wb') as file:
                file.write(file_content)

        await session.close()
             
def attack():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(client(1))
    loop.close()
    
threads = []
for i in range(850): # breaks at 900
    thread = threading.Thread(target=attack)
    thread.start()
    threads.append(thread)

count = 0
for thread in threads:
    thread.join()
    count += 1
    print(count, "threads joined")

In [ ]:
import threading
import asyncio
import websockets
import json

async def receive_file(websocket, file_count):
    for i in range(file_count):
        try:
            # Receive data from the WebSocket connection
            data_received = await websocket.recv()
            # Convert received data to a Python dictionary from JSON format
            data_received = json.loads(data_received.replace("'", '"'))
            # Extract the filename to save from the received data
            filename_to_save = data_received["filename"]
            # Convert the content to bytes for writing to a file
            file_content = bytes(data_received["content"])
            # Write the content to a file with the specified filename
            with open(f"{filename_to_save}", 'wb') as file:
                file.write(file_content)
        except websockets.ConnectionClosedOK:
            break


async def client(file_count):
    # Establish a WebSocket connection to the specified address
    async with websockets.connect('ws://localhost:8765') as websocket:
        # Call the send_file function to send the specified file
        await websocket.send(str(file_count))
        await receive_file(websocket, file_count)

def attack():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(client(1))
    loop.close()
    
threads = []
for i in range(175): # breaks in 180
    thread = threading.Thread(target=attack)
    thread.start()
    threads.append(thread)

count = 0
for thread in threads:
    thread.join()
    count += 1
    print(count, "threads joined")